# ngly1-ntwk

## Purpose: 
* explore NGLY1 network, assembled by Nuria (~5,000 nodes, ~40,000 edges)
* figure out how to best represent / store network data in python
* calculate connections between nodes (node pathways)
    * filter data
    * aggregate / summarize pathways
    * calc connections between terms
* test out ways to visualize network

### Authorship notes
Laura Hughes, lhughes@scripps.edu

based off of data assembled by Nuria Queralt Rosinach

January 2018

In [1]:
import pandas as pd
import matplotlib as mp
import string
from collections import OrderedDict, defaultdict

In [6]:
nodes = pd.read_csv('~/GitHub/ngly1/neo4j-community-3.0.3/import/ngly1/ngly1_concepts.tsv')
nodes.head()

,id:ID,:LABEL,preflabel,synonyms:IGNORE,description
0,ZFIN:ZDB-GENE-051023-7,GENE,uba52,NaN,None
1,UBERON:0002107,ANAT,liver,NaN,None
2,MGI:106594,GENE,Gk,NaN,None
3,UBERON:0004720,ANAT,cerebellar vermis,NaN,None
4,FlyBase:FBgn0033807,GENE,AQP,NaN,None


In [5]:
edges = pd.read_csv('~/GitHub/ngly1/neo4j-community-3.0.3/import/ngly1/ngly1_statements.tsv')
edges.head()

,:START_ID,:TYPE,:END_ID,reference_uri,reference_supporting_text,reference_date,property_label,property_description,property_uri
0,ZFIN:ZDB-GENE-051023-7,RO:0002206,UBERON:0002107,NaN,NaN,NaN,expressed in,None,http://purl.obolibrary.org/obo/RO_0002206
1,MGI:106594,RO:0002206,UBERON:0004720,NaN,NaN,NaN,expressed in,None,http://purl.obolibrary.org/obo/RO_0002206
2,FlyBase:FBgn0033807,RO:0002200,FBcv:0000349,https://www.ncbi.nlm.nih.gov/pubmed/20220848,NaN,NaN,has phenotype,None,http://purl.obolibrary.org/obo/RO_0002200
3,NCBIGene:824852,RO:HOM0000017,FlyBase:FBgn0033010,NaN,NaN,NaN,in orthology relationship with,None,http://purl.obolibrary.org/obo/RO_HOM0000017
4,NCBIGene:470634,RO:HOM0000020,MGI:1342541,NaN,NaN,NaN,in 1 to 1 orthology relationship with,None,http://purl.obolibrary.org/obo/RO_HOM0000020


In [25]:
edges.groupby(['property_label']).count()

,:START_ID,:TYPE,:END_ID,reference_uri,reference_supporting_text,reference_date,property_description,property_uri
property_label,,,,,,,,
None,1274,1274,1274,1011,0,0,1274,1274
Source,49,49,49,49,0,0,49,49
colocalizes with,60,60,60,60,0,0,60,60
contributes to,1,1,1,1,0,0,1,1
derives_from,9,9,9,0,0,0,9,9
enables,582,582,582,582,0,0,582,582
exact match,2,2,2,0,0,0,0,0
expressed in,2303,2303,2303,0,0,0,2303,2303
genetically interacts with,12,12,12,12,0,0,12,12


In [59]:
edges2 = edges.sort_values(':START_ID')

edges2.head()

edges2 = edges2[0:11][[':START_ID', ':END_ID']]
edges2.set_index(':START_ID')

edges2

,:START_ID,:END_ID
25373,:.well-known/genid/H_AQP1_C01_TM1B,MGI:103201
14343,:.well-known/genid/H_AQP1_C01_TM1B,MGI:5692545
30776,:.well-known/genid/MGI101922-VLU-MGI102724-VLU...,MGI:103201
17460,:.well-known/genid/MGI101922-VLU-MGI102724-VLU...,MGI:1100851
22606,:.well-known/genid/MGI102702-VLU-MGI103201-VLU...,MGI:1338871
12601,:.well-known/genid/MGI102702-VLU-MGI103201-VLU...,MGI:103201
35552,:.well-known/genid/MGI103076-VLU-MGI103198-VLU...,MGI:103201
32547,:.well-known/genid/MGI103201-VL,MGI:103201
19824,:.well-known/genid/MGI103201-VLU-MGI107387-VLU...,MGI:107387
23297,:.well-known/genid/MGI103201-VLU-MGI107387-VLU...,MGI:103201


In [65]:
dd = defaultdict(list)
df= edges2.to_dict()

edges2
df


{':END_ID': {8180: 'MGI:103201',
  12601: 'MGI:103201',
  14343: 'MGI:5692545',
  17460: 'MGI:1100851',
  19824: 'MGI:107387',
  22606: 'MGI:1338871',
  23297: 'MGI:103201',
  25373: 'MGI:103201',
  30776: 'MGI:103201',
  32547: 'MGI:103201',
  35552: 'MGI:103201'},
 ':START_ID': {8180: ':.well-known/genid/MGI5692545GENO0000137',
  12601: ':.well-known/genid/MGI102702-VLU-MGI103201-VLU-MGI104576-VLU-MGI104697-VLU-MGI104960-VLU-MGI105370-VLU-MGI105959-VLU-MGI107560-VLU-MGI107690-VLU-MGI107726-VLU-MGI108559-VLU-MGI1095407-VLU-MGI1096573-VLU-MGI1098550-VLU-MGI1196391-VLU-MGI1276116-VLU-MGI1277238-VLU-MGI1316658-VLU-MGI1330860-VLU-MGI1336167-VLU-MGI1338871-VLU-MGI1342290-VLU-MGI1343180-VLU-MGI1353576-VLU-MGI1858414-VLU-MGI1889341-VLU-MGI1891717-VLU-MGI1891830-VLU-MGI1913429-VLU-MGI1913855-VLU-MGI1914080-VLU-MGI1914269-VLU-MGI1914324-VLU-MGI1915134-VLU-MGI1915467-VLU-MGI1917452-VLU-MGI1917695-VLU-MGI1918180-VLU-MGI1918195-VLU-MGI1918940-VLU-MGI1919174-VLU-MGI1919318-VLU-MGI1919429-VLU-MGI19

In [3]:
list(string.ascii_letters)

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']